## Exercícios

> Retirados de [learn-python: sqlalchemy_orm-questions](https://aviadr1.github.io/learn-advanced-python/11_db_access/exercise/sqlalchemy_orm-questions.html).

#### Q1.

Baixa e extraia o arquivo compactado com o banco de dados [Chinook database](https://www.sqlitetutorial.net/sqlite-sample-database/). Salve o arquivo `chinook.db` na mesma pasta deste script.
* Link para baixar: http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip

<img width=500 src=https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg>


#### Q2.

Leia o código e os comentários das células a seguir para entender como acessamos os modelos ORM de um banco já existente.

In [1]:
from sqlalchemy import create_engine, text, MetaData, select # adicionei select devido a erro na célula 2
from sqlalchemy.orm import Session

engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)

### extrai as classes da base de dados Chinook
metadata = MetaData()
metadata.reflect(engine)

# O metadata tem informações sobre as tabelas
# que serão usadas para criar os modelos ORM
for table_name, table in metadata.tables.items():
    print(table_name)
    print(table.columns.keys())
    print(table.columns.items())
    print('-'*25)

### configura o objeto Base mapeando os modelos ORM das tabelas
from sqlalchemy.ext.automap import automap_base
Base = automap_base(metadata=metadata)
Base.prepare()

# o objeto Base tem os modelos ORM que podemos usar
# para manipular o banco de dados
print(Base.classes.items())

albums
['AlbumId', 'Title', 'ArtistId']
[('AlbumId', Column('AlbumId', INTEGER(), table=<albums>, primary_key=True, nullable=False)), ('Title', Column('Title', NVARCHAR(length=160), table=<albums>, nullable=False)), ('ArtistId', Column('ArtistId', INTEGER(), ForeignKey('artists.ArtistId'), table=<albums>, nullable=False))]
-------------------------
artists
['ArtistId', 'Name']
[('ArtistId', Column('ArtistId', INTEGER(), table=<artists>, primary_key=True, nullable=False)), ('Name', Column('Name', NVARCHAR(length=120), table=<artists>))]
-------------------------
customers
['CustomerId', 'FirstName', 'LastName', 'Company', 'Address', 'City', 'State', 'Country', 'PostalCode', 'Phone', 'Fax', 'Email', 'SupportRepId']
[('CustomerId', Column('CustomerId', INTEGER(), table=<customers>, primary_key=True, nullable=False)), ('FirstName', Column('FirstName', NVARCHAR(length=40), table=<customers>, nullable=False)), ('LastName', Column('LastName', NVARCHAR(length=20), table=<customers>, nullable=F

In [2]:
# A seguir um exemplo de query na tabela Albums
# usamos o objeto Base para acessar cada modelo ORM.

session = Session(engine)
res = session.scalars(select(Base.classes.albums))
first_album = res.first()
print(first_album.AlbumId, first_album.Title)

1 For Those About To Rock We Salute You


#### Q3. 
Com base nos códigos anteriores realize as operações solicitadas nas células a seguir:


In [3]:
### Imprima os três primeiros registros da tabela tracks
tracks = Base.classes.tracks
res = session.scalars(select(tracks).limit(3))
for track in res:
    print(track.TrackId, track.Name)

1 For Those About To Rock (We Salute You)
2 Balls to the Wall
3 Fast As a Shark


/tmp/ipykernel_17167/1612725063.py:3: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  res = session.scalars(select(tracks).limit(3))


In [4]:
### Imprima o nome da faixa e o título do álbum das primeiras 20 faixas na tabela tracks.
from sqlalchemy import select, join

tracks = Base.classes.tracks
albums = Base.classes.albums

stmt = (
    select(tracks.Name, albums.Title)
    .join(albums, tracks.AlbumId == albums.AlbumId)
    .limit(20)
)

res = session.execute(stmt).all()
for track_name, album_title in res:
    print(f"Faixa: {track_name} - Álbum: {album_title}")

Faixa: For Those About To Rock (We Salute You) - Álbum: For Those About To Rock We Salute You
Faixa: Put The Finger On You - Álbum: For Those About To Rock We Salute You
Faixa: Let's Get It Up - Álbum: For Those About To Rock We Salute You
Faixa: Inject The Venom - Álbum: For Those About To Rock We Salute You
Faixa: Snowballed - Álbum: For Those About To Rock We Salute You
Faixa: Evil Walks - Álbum: For Those About To Rock We Salute You
Faixa: C.O.D. - Álbum: For Those About To Rock We Salute You
Faixa: Breaking The Rules - Álbum: For Those About To Rock We Salute You
Faixa: Night Of The Long Knives - Álbum: For Those About To Rock We Salute You
Faixa: Spellbound - Álbum: For Those About To Rock We Salute You
Faixa: Balls to the Wall - Álbum: Balls to the Wall
Faixa: Fast As a Shark - Álbum: Restless and Wild
Faixa: Restless and Wild - Álbum: Restless and Wild
Faixa: Princess of the Dawn - Álbum: Restless and Wild
Faixa: Go Down - Álbum: Let There Be Rock
Faixa: Dog Eat Dog - Álbum: Le

In [5]:
### Imprima as 10 primeiras vendas de faixas da tabela invoice_items
### Para essas 10 primeiras vendas, imprima os nomes das faixas vendidas e a quantidade vendida.
invoice_items = Base.classes.invoice_items
tracks = Base.classes.tracks

stmt = (
    select(invoice_items.Quantity, tracks.Name)
    .join(tracks, invoice_items.TrackId == tracks.TrackId)
    .limit(10)
)

res = session.execute(stmt).all()
for quantity, track_name in res:
    print(f"Faixa: {track_name} - Quantidade vendida: {quantity}")

Faixa: Balls to the Wall - Quantidade vendida: 1
Faixa: Restless and Wild - Quantidade vendida: 1
Faixa: Put The Finger On You - Quantidade vendida: 1
Faixa: Inject The Venom - Quantidade vendida: 1
Faixa: Evil Walks - Quantidade vendida: 1
Faixa: Breaking The Rules - Quantidade vendida: 1
Faixa: Dog Eat Dog - Quantidade vendida: 1
Faixa: Overdose - Quantidade vendida: 1
Faixa: Love In An Elevator - Quantidade vendida: 1
Faixa: Janie's Got A Gun - Quantidade vendida: 1


In [6]:
### Imprima os nomes das 10 faixas mais vendidas e quantas vezes foram vendidas.
from sqlalchemy import func, desc

stmt = (
    select(tracks.Name, func.sum(invoice_items.Quantity).label("total_vendido"))
    .join(invoice_items, tracks.TrackId == invoice_items.TrackId)
    .group_by(tracks.TrackId)
    .order_by(desc("total_vendido"))
    .limit(10)
)

res = session.execute(stmt).all()
for track_name, total in res:
    print(f"Faixa: {track_name} - Total vendido: {total}")

Faixa: Balls to the Wall - Total vendido: 2
Faixa: Inject The Venom - Total vendido: 2
Faixa: Snowballed - Total vendido: 2
Faixa: Overdose - Total vendido: 2
Faixa: Deuces Are Wild - Total vendido: 2
Faixa: Not The Doctor - Total vendido: 2
Faixa: Por Causa De Você - Total vendido: 2
Faixa: Welcome Home (Sanitarium) - Total vendido: 2
Faixa: Snowblind - Total vendido: 2
Faixa: Cornucopia - Total vendido: 2


In [7]:
### Quem são os 10 artistas que mais venderam?
### dica: você precisa juntar as tabelas invoice_items, tracks, albums e artists
artists = Base.classes.artists
albums = Base.classes.albums
tracks = Base.classes.tracks
invoice_items = Base.classes.invoice_items

stmt = (
    select(artists.Name, func.sum(invoice_items.Quantity).label("total_vendido"))
    .join(albums, albums.ArtistId == artists.ArtistId)
    .join(tracks, tracks.AlbumId == albums.AlbumId)
    .join(invoice_items, invoice_items.TrackId == tracks.TrackId)
    .group_by(artists.ArtistId)
    .order_by(desc("total_vendido"))
    .limit(10)
)

res = session.execute(stmt).all()
for artist_name, total in res:
    print(f"Artista: {artist_name} - Total vendido: {total}")

Artista: Iron Maiden - Total vendido: 140
Artista: U2 - Total vendido: 107
Artista: Metallica - Total vendido: 91
Artista: Led Zeppelin - Total vendido: 87
Artista: Os Paralamas Do Sucesso - Total vendido: 45
Artista: Deep Purple - Total vendido: 44
Artista: Faith No More - Total vendido: 42
Artista: Lost - Total vendido: 41
Artista: Eric Clapton - Total vendido: 40
Artista: R.E.M. - Total vendido: 39
